In [1]:
from spacy.lang.ru import Russian
import spacy
import pymorphy2
import string

import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords

from typing import List, TextIO

from collections import deque 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sshghoult\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# Spacy Tokenization example
sample_text = "Съешь же этих мягких! французских, булок... да выпей чаю!?"

nlp = spacy.load('ru_core_news_sm')

In [3]:
# morph = pymorphy2.MorphAnalyzer()
# token_lemmas = [morph.parse(token)[0].normal_form for token in token_list]
# token_lemmas

In [4]:

    # это отвратительнейший дизайн архитектуры, нужно придумать что-то более адекватное
    
    # в идеале - имитация генератора
    
    # next -> следующий из внутреннего буфера -> если в буфере пусто, то взять чанк, очистить и положить в буфер
    # как отделить выделение по предложениям от непрерывного в интерфейсах?
    # взятие чанка overwritten, метод очистки в базовом классе
    
    # Паттерн сборки с возвращением self и сохранением буффера

In [ ]:
class BaseLazyTextPreprocessor(object):
    """Ленивый препроцессор для очистки текста перед работой с моделями"""
    def __init__(self, fp: TextIO, nlp, stopwords):
        self.src = fp
        self.nlp = nlp
        self.stopwords = stopwords
        self.stoppunct = set(string.punctuation).union({'...', '?!'})
        self.endpunct = {'.', '!', '?'}
        
    
    def preprocess(self, chunk: str):
        doc = self.nlp(chunk)
        cleaned = [token.lemma_ for token in doc if token.lemma_ not in self.stopwords and token.lemma_ != " " 
                   and token.lemma_ not in self.stoppunct]
        return cleaned


In [11]:
with open('tmp.txt', 'r', encoding='utf-8') as fp:
    LTP = BaseLazyTextPreprocessor(fp, nlp, stopwords.words("russian"))
    print(LTP.preprocess(fp.read()))

['съешь', 'мягкий', 'французский', 'булка', 'выпей', 'чаять']


In [ ]:
class CollocationLazyTextProcessor(BaseLazyTextPreprocessor):
    def __init__(self, fp: TextIO, nlp, stopwords):
        super().__init__(fp, nlp, stopwords)
        self.clean_buffer = deque()
        
    def __next__(self):
        if len(self.clean_buffer) == 0:
            self.__load_buffer()
        return self.clean_buffer.pop()
    
    def __iter__(self):
        return self
    
    def __load_buffer(self, size=5):
        pass
        # вычитать предложение, отправить его в препроцесс, положить в буффер
        for _ in range(size):
            subbuffer = []
            ptr = ''
            while ptr not in self.stoppunct:
                ptr = self.src.read(1)
                subbuffer.append(ptr)
            cleaned = self.preprocess(''.join(subbuffer))
            if cleaned:
                self.clean_buffer.appendleft(cleaned)

In [ ]:
with open('tmp.txt', 'r', encoding='utf-8') as fp:
    LTP = CollocationLazyTextProcessor(fp, nlp, stopwords.words("russian"))
    for k in LTP:
        print(k)